In [9]:
!pip install apt

ERROR: Could not find a version that satisfies the requirement apt (from versions: none)
ERROR: No matching distribution found for apt


In [10]:
!pip install Selenium

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [1]:

import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException


chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
def comments_scraping(news_url,wd):
    title=wd.find_element_by_id('articleTitle').text
    datetime=wd.find_element_by_class_name('t11').text
    print('뉴스:',[title,datetime,news_url])

    try:
        btn_view=wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
        #print('[댓글 더보기]',end='')
        time.sleep(1)

        while True:
              wd.find_element_by_class_name('u_cbox_page_more').click()
              #print('[더보기]',end='')
              time.sleep(1)

    except:
            pass


  #btn_reply_list=wd.find_elements_by_class_name('u_cbox_btn_reply')
  #for btn_reply in btn_reply_list:
  #  btn_reply.send_keys('\n')
  #  print('[답글]',end='')
  #  time.sleep(1)

  #print('[댓글 스크래핑]')
    comments_idx=0
    comments_df=pd.DataFrame(columns=['Title','date','Contents','Datetime','URL'])

    comments=wd.find_elements_by_class_name('u_cbox_comment_box')
    for comment in comments:
            try:
                date=comment.find_element_by_class_name('u_cbox_date').text
                contents=comment.find_element_by_class_name('u_cbox_contents').text
                #print(f' 댓글 #{comments_idx+1}:',[contents,date,news_url])
                comments_df.loc[comments_idx]=[title, datetime, contents,date,news_url]
                comments_idx+=1
                
            except NoSuchElementException:
                #print('[삭제되거나 부적절한 댓글]')
                continue

    return comments_df

In [4]:
#191120~191130:검색결과 약 14,600,000개 (0.38초) 15->8
#191201~191231:검색결과 약 37,300,000개 (0.26초) 37->19
#200101~200131:검색결과 약 52,000,000개 (0.31초)  52->26
#200201~200229:검색결과 약 37,700,000개 (0.31초)  38->19
#200301~200331:검색결과 약 136,000,000개 (0.34초) 136->68
#200401~200430:검색결과 약 140,000,000개 (0.32초) 140->70
#200501~200531:검색결과 약 73,300,000개 (0.30초) 73->37
#200601~200630:검색결과 약 107,000,000개 (0.35초) 107->54
#200701~200731:검색결과 약 99,500,000개 (0.32초) 100->50
#200801~200831:검색결과 약 129,000,000개 (0.34초) 129->65
#200901~200930:검색결과 약 62,800,000개 (0.34초) 63->32
#201001~201031:검색결과 약 165,000,000개 (0.38초) 165->83
#201101~201130:검색결과 약 81,800,000개 (0.47초) 82->42
#201201~201217:검색결과 약 769,000개 (0.40초)   ->4


def scraping():
    wd=webdriver.Chrome('chromedriver',options=chrome_options)
    wd.implicitly_wait(3)
  
    wd.execute_script('window.open("about:blank","_blank");')
    tabs=wd.window_handles
    wd.switch_to.window(tabs[0])
    query='셀트리온'  #input('검색어 입력: ')
    sort = '2'   #관련도순=0  최신순=1  오래된순=2
    s_date = '2021.01.01'  #input("시작날짜 입력(20xx.xx.xx):")  
    e_date = '2020.05.10'    #input("끝날짜 입력(20xx.xx.xx):")
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    search_url="https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=0" 
    wd.get(search_url)
    print(search_url)


    comments_df=pd.DataFrame()
    
    page=0
    while 1:
        article_list=wd.find_elements_by_css_selector('ul.list_news > li.bx')
        len(article_list)
        #print("list ",article_list)
        target_naver_news_href=[]

        for this_article in article_list:
            naver_news_check=this_article.find_elements_by_css_selector('div.news_wrap.api_ani_send > a')
            print(naver_news_check)
            len(naver_news_check)
            for this_cand in naver_news_check:
                target_naver_news_href.append(this_cand.get_attribute('href'))
        print(target_naver_news_href)
        print(len(target_naver_news_href))
    
        
        while target_naver_news_href!=[]:
            try:
                wd.switch_to.window(tabs[1])
                get_url=target_naver_news_href.pop(0)
                wd.get(get_url)
                
                print("get_url : ",get_url)

                df=comments_scraping(get_url,wd)
                #comments_df.loc[comments_idx]=[query]
                comments_df=pd.concat([df,comments_df])
                

            except:
                continue
            
        
  
        try:
            wd.switch_to.window(tabs[0])
            nextpage=wd.find_element_by_class_name('btn_next').get_attribute('href')
            wd.get(nextpage)
            print('[다음페이지]')
            time.sleep(1)
            page+=1

        except:
            break      

    return comments_df

In [ ]:
comments_df=scraping()

https://search.naver.com/search.naver?where=news&query=셀트리온&sort=2&ds=2021.01.01&de=2020.05.10&nso=so%3Ar%2Cp%3Afrom20210101to20200510%2Ca%3A&start=0
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="7f73d263-24c4-4d94-a109-9ba22314bf63")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="61cffba5-20c8-4df5-bc95-358b1ec53ab4")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="8ee85ae6-2b98-4dd1-919b-30ba74e0d1bf")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="79601ae8-4ce3-46ce-9c69-c5235c0e5e24")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="76862004-3733-4167-a3e0-6bc09db47c2b")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="a5f335ec-14a3-467c-b2

get_url :  http://www.smartfn.co.kr/view.php?ud=202005101739544560861beb2d9d_46
get_url :  http://www.dt.co.kr/contents.html?article_no=2020051102101031102001&ref=naver
get_url :  http://news.mk.co.kr/newsRead.php?no=476701&year=2020
get_url :  http://www.dynews.co.kr/news/articleView.html?idxno=499993
get_url :  https://www.yeongnam.com/web/view.php?key=20200510010001196
get_url :  http://www.fintechpost.co.kr/news/articleView.html?idxno=113292
[다음페이지]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="e1850439-6199-4a23-9190-7e66ee064734")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="b55a227f-9d9f-485b-a091-679e48b85ac5")>]
[]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="9d3e7ab5-2cfc-4363-807f-167e08b862b2")>]
[<selenium.webdriver.remote.webelement.WebElement (session="e421c54e0ffb824c37db85ba69ce22df", element="